In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-06 01:19:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-09-06 01:19:49 (16.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(len(text))

1115394


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# character level tokenizing
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(context, target) #these are the examples

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # num of sequences we process at once with parallel processing
block_size = 8 # max context length

def gen_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x,y

xb,yb = gen_batch('train')
print(xb.shape, xb, yb.shape, yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(context.tolist(), target)

torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
[24] tensor(43)
[24, 43] tensor(58)
[24, 43, 58] tensor(5)
[24, 43, 58, 5] tensor(57)
[24, 43, 58, 5, 57] tensor(1)
[24, 43, 58, 5, 57, 1] tensor(46)
[24, 43, 58, 5, 57, 1, 46] tensor(43)
[24, 43, 58, 5, 57, 1, 46, 43] tensor(39)
[44] tensor(53)
[44, 53] tensor(56)
[44, 53, 56] tensor(1)
[44, 53, 56, 1] tensor(58)
[44, 53, 56, 1, 58] tensor(46)
[44, 53, 56, 1, 58, 46] tensor(39)
[44, 53, 56, 1, 58, 46, 39] tensor(58)
[44, 53, 56, 1, 58, 46, 39, 58] tensor(1)
[52] tensor(58)
[52, 58] tensor(1)
[52, 58, 1] tensor(58)
[52, 58, 1, 58] tensor(46)
[52, 58, 1, 58, 46] tensor(39)
[52, 58, 1, 58, 46, 39] tensor(58)
[52, 58, 1

In [ ]:
#bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as f
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #lookup table for next token

  def forward(self, idx, targets=None):  #idx and targets are both (B,T) tensors
    logits = self.token_embedding_table(idx) # (B,T,C) batch=4, time=8, channel=65 tensor

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T) #cross entropy has to have C as second dimension of logits
      loss = f.cross_entropy(logits, targets) # finding the loss of logits with respect to target

    return logits, loss

  def generate(self, idx, max_new_tokens): #extends (B,T) by max_new_tokens in the time direction
    #idx is a (B,T) array of indices in current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx)
      #focus only on last time step
      logits = logits[:, -1, :] # becomes (B,C) tensor, using last element in time dimension since that is the prediction for what is next
      # apply softmax to get probabilities
      probs = f.softmax(logits, dim=-1) # (B,C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      #append sampled index to running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

#0 kicks off the sequence, generate 100 tokens
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #pytorch optimizer

In [ ]:
batch_size = 32
for steps in range(10000):
  xb, yb = gen_batch('train') # get batch
  logits, loss = m(xb,yb) #evaluate loss
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5727508068084717


In [ ]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


In [ ]:
torch.manual_seed(1377)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    # xprev = x[b,:t+1] # (t,C)
    # xbow[b,t] = torch.mean(xprev, 0) # mean x[b,i] where i<=t
    wei = torch.tril(torch.ones(T,T))
    wei = wei / wei.sum(1,keepdim=True) # matrix mult to get the avg (makes multipliers/weights)
    xbow = wei @ x # (T, T) * (B, T, C), pytorch makes batch dim for wei, (B, T, T) * (B, T, C) = (B, T, C)

#use matrix multiplication for efficiency


In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2,-1) #transpose last and second to last, (B,T,16) @ (B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf")) #all elements where tril = 0 become -inf in wei, remove for all nodes communicate with each other, like sentiment analysis
wei = f.softmax(wei, dim=-1) # normalizes, makes the same weighted multiplier matrix, research paper 3.2.1 (1)
#softmax exponentiates (to the power of smth) each element, and divides by the sum
#in our case it just uses the 0s, makes them 1s and then normalizes them

v = value(x) # x is private info, v is what we use to aggregate
out = wei @ v
out.shape

#self attention: each node gives a query and a key query: what am I looking for key: what do I contain wei = my query dot product with all other keys, learn more where key and query are aligned

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from google.colab import drive
drive.mount('/content/drive')

#hyperparams
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
# if torch.backends.mps.is_available():
#     torch.multiprocessing.set_start_method('spawn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def gen_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

@torch.no_grad() #tell pytorch not to track gradients for backprop
def estimate_loss():
    out = {}
    model.eval() #set to eval phase
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = gen_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() #set back to train phase
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,16)
        q = self.query(x) # (B,T,16)

        #compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5
        #transpose last and second to last, (B,T,16) @ (B,16,T) -> (B,T,T), sqrt(C) so that we dont favor just one node too much
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf")) #all elements where tril = 0 become -inf in wei, remove for all nodes communicate with each other, like sentiment analysis
        wei = F.softmax(wei, dim=-1) # normalizes, makes the same weighted multiplier matrix, research paper 3.2.1 (1)
        #softmax exponentiates (to the power of smth) each element, and divides by the sum
        #in our case it just uses the 0s, makes them 1s and then normalizes them
        wei = self.dropout(wei) # (B,T,T), randomly prevents some nodes from communicating to prevent overfitting
        v = self.value(x) # x is private info, v is what we use to aggregate
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """" Multi heads of self attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ linear layer followed by a nonlinearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(), #nonlinearity
            nn.Linear(4* n_embd, n_embd), #projection
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block with communication and computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) #self attention head
        self.ffwd = FeedForward(n_embd) #feed forward layer
        self.ln1 = nn.LayerNorm(n_embd) #layer norm
        self.ln2 = nn.LayerNorm(n_embd) #layer norm

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # (B,T,C), apply to self attention head
        x = x + self.ffwd(self.ln2(x)) # (B,T,C), apply to feed forward layer
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) #lookup table for next token
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #lookup table for position
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)]) #3 transformer blocks
        self.ln_f = nn.LayerNorm(n_embd) #layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) #linear layer to get logits

    def forward(self, idx, targets=None):  #idx and targets are both (B,T) tensors
        B,T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,C=vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T) #cross entropy has to have C as second dimension of logits
            loss = F.cross_entropy(logits, targets) # finding the loss of logits with respect to target

        return logits, loss

    def generate(self, idx, max_new_tokens): #extends (B,T) by max_new_tokens in the time direction
        #idx is a (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            #only use at max block_size tokens
            idx_cond = idx[:, -block_size:]
            #get the predictions
            logits, loss = self(idx_cond)
            #focus only on last time step
            logits = logits[:, -1, :] # becomes (B,C) tensor, using last element in time dimension since that is the prediction for what is next
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            #append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate) #pytorch optimizer

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = gen_batch('train') # get batch
    logits, loss = m(xb,yb) #evaluate loss
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

torch.save(m.state_dict(), '/content/drive/MyDrive/model.pth')
#context = torch.zeros((1,1), dtype=torch.long, device=device)
#print(decode(m.generate(context, max_new_tokens=300)[0].tolist()))



# Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
# There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
# Each example across batch dimension is of course processed completely independently and never "talk" to each other
# In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
# "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
# "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda
step 0: train loss 4.2849, val loss 4.2823
step 500: train loss 2.0099, val loss 2.0961
step 1000: train loss 1.6017, val loss 1.7816
step 1500: train loss 1.4414, val loss 1.6390
step 2000: train loss 1.3430, val loss 1.5733
step 2500: train loss 1.2798, val loss 1.5327
step 3000: train loss 1.2281, val loss 1.5107
step 3500: train loss 1.1833, val loss 1.4902
step 4000: train loss 1.1475, val loss 1.4842
step 4500: train loss 1.1112, val loss 1.4822


In [ ]:
md = BigramLanguageModel()
md.load_state_dict(torch.load('model.pth'))
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=300)[0].tolist()))


Yows!

GORK:
If OFFirpler Engmicemee bents!than meplares uch
Nowald werench handill, aveand, that che them have
What wom thath astus your an, memar the elowie.

RUCEO:
WAtll watlle mun:
Orut sileis han hes haveve in bevewele me
Your noth ot hom kispe youod?
Hpesener's yencaver stoncle; thus wir you,


In [ ]:
md = BigramLanguageModel()
md.to(device)
md.load_state_dict(torch.load('/content/drive/MyDrive/model.pth'))
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(md.generate(context, max_new_tokens=100000)[0].tolist()))